In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import descartes
from shapely.geometry import Point, Polygon
%matplotlib inline

In [4]:
uni_df = pd.read_csv("ef2004cp_rv.csv")

In [5]:
uni_df

,UNITID,EFCIPLEV,CIPCODE,LINE,SECTION,LSTUDY,XEFRAC01,EFRACE01,XEFRAC02,EFRACE02,...,XEFRAC20,EFRACE20,XEFRAC21,EFRACE21,XEFRAC22,EFRACE22,XEFRAC23,EFRACE23,XEFRAC24,EFRACE24
0,100654,101,13.0,29,3,1,R,6,R,21,...,R,3,R,3,R,234,Z,0,R,1333
1,100654,102,13.0,99,3,2,R,4,R,12,...,R,2,R,3,R,41,Z,0,R,870
2,100654,103,13.0,99,3,3,R,4,R,11,...,R,2,R,3,R,41,Z,0,R,865
3,100654,104,13.0,99,3,4,Z,0,Z,0,...,Z,0,R,1,R,1,Z,0,R,160
4,100654,105,13.0,99,3,5,R,4,R,11,...,R,2,R,2,R,40,Z,0,R,705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112863,445717,622,52.0,8,1,22,R,23,R,25,...,R,21,R,128,R,211,R,164,R,607
112864,445717,623,52.0,6,1,23,R,23,R,25,...,R,21,R,128,R,211,R,164,R,607
112865,445717,624,52.0,1,1,24,R,1,R,0,...,Z,0,R,1,Z,0,R,6,R,9
112866,445717,625,52.0,3,1,25,R,22,R,25,...,R,21,R,127,R,211,R,158,R,598


"EFRACE15" is total Men
"EFRACE16" is total Women
"EFCIPLEV" is 202 for undergraduate engineering students

In [6]:
uni_df1 = uni_df[['UNITID', 'EFCIPLEV', 'EFRACE15', 'EFRACE16']]

In [7]:
uni_df1.head()

,UNITID,EFCIPLEV,EFRACE15,EFRACE16
0,100654,101,452,881
1,100654,102,366,504
2,100654,103,366,499
3,100654,104,72,88
4,100654,105,294,411


In [8]:
uni_df2 = uni_df1.loc[uni_df1["EFCIPLEV"].isin([202])]

In [9]:
uni_df2

,UNITID,EFCIPLEV,EFRACE15,EFRACE16
22,100654,202,373,95
128,100663,202,389,90
248,100706,202,1097,253
457,100751,202,1114,260
660,100858,202,2708,462
...,...,...,...,...
112277,443128,202,9,5
112354,443562,202,35,9
112614,444848,202,20,7
112661,445072,202,1,0


In [10]:
uni_df3 = uni_df2[['UNITID', 'EFRACE15', 'EFRACE16']]

In [11]:
uni_df4 = uni_df3.groupby(['UNITID']).sum()

In [12]:
uni_df4

,EFRACE15,EFRACE16
UNITID,,
100654,373,95
100663,389,90
100706,1097,253
100751,1114,260
100858,2708,462
...,...,...
443128,9,5
443562,35,9
444848,20,7


In [13]:
uni_df5 = uni_df4.reset_index()
uni_df5

,UNITID,EFRACE15,EFRACE16
0,100654,373,95
1,100663,389,90
2,100706,1097,253
3,100751,1114,260
4,100858,2708,462
...,...,...,...
570,443128,9,5
571,443562,35,9
572,444848,20,7
573,445072,1,0


In [14]:
uni_df5['W/M_Ratio'] = uni_df5['EFRACE16']/uni_df5['EFRACE15']
uni_df5

,UNITID,EFRACE15,EFRACE16,W/M_Ratio
0,100654,373,95,0.254692
1,100663,389,90,0.231362
2,100706,1097,253,0.230629
3,100751,1114,260,0.233393
4,100858,2708,462,0.170606
...,...,...,...,...
570,443128,9,5,0.555556
571,443562,35,9,0.257143
572,444848,20,7,0.350000
573,445072,1,0,0.000000


# Bring in University Location Data

In [15]:
UniLoc = pd.read_csv("UniversityID_Lat_Long_ONLY.csv")

In [16]:
UniLoc

,UNITID,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE
0,100654,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.505697,-86.799345
2,100690,Amridge University,Montgomery,AL,36117-3553,32.362609,-86.174010
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,34.724557,-86.640449
4,100724,Alabama State University,Montgomery,AL,36104-0271,32.364317,-86.295677
...,...,...,...,...,...,...,...
6689,48511113,Georgia Military College - Eastman,Eastman,GA,310236109,NaN,NaN
6690,48616901,American College of Barbering - Florence,Florence,KY,410421801,NaN,NaN
6691,49005401,HCI College - Fort Lauderdale Campus,Fort Lauderdale,FL,33309,NaN,NaN
6692,49146401,ABC Adult School - Cabrillo Lane,Cerritos,CA,907037748,NaN,NaN


JOIN!

In [17]:
df = pd.merge(uni_df5, UniLoc, on = "UNITID")
df

,UNITID,EFRACE15,EFRACE16,W/M_Ratio,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE
0,100654,373,95,0.254692,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502
1,100663,389,90,0.231362,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.505697,-86.799345
2,100706,1097,253,0.230629,University of Alabama in Huntsville,Huntsville,AL,35899,34.724557,-86.640449
3,100751,1114,260,0.233393,The University of Alabama,Tuscaloosa,AL,35487-0100,33.211875,-87.545978
4,100858,2708,462,0.170606,Auburn University,Auburn,AL,36849,32.599378,-85.488258
...,...,...,...,...,...,...,...,...,...,...
545,436827,13,0,0.000000,University of Connecticut-Avery Point,Groton,CT,06340-6097,41.316419,-72.065336
546,436836,7,0,0.000000,University of Connecticut-Stamford,Stamford,CT,06901-2315,41.056081,-73.542206
547,441982,112,89,0.794643,Franklin W Olin College of Engineering,Needham,MA,02492-1200,42.293527,-71.263378
548,443128,9,5,0.555556,International Technological University,Santa Clara,CA,95054,37.377192,-121.963426


Change to GEO Dataframe

In [18]:
crs = {'init':'epsg:4326'}
geometry = [Point(xy) for xy in zip(df["LONGITUDE"], df["LATITUDE"])]
geometry[:3]
geo_df = gpd.GeoDataFrame(df, crs = crs, geometry = geometry)
geo_df.head()

C:\tools\Anaconda3\envs\GeoSpacial\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,UNITID,EFRACE15,EFRACE16,W/M_Ratio,INSTNM,CITY,STABBR,ZIP,LATITUDE,LONGITUDE,geometry
0,100654,373,95,0.254692,Alabama A & M University,Normal,AL,35762,34.783368,-86.568502,POINT (-86.56850 34.78337)
1,100663,389,90,0.231362,University of Alabama at Birmingham,Birmingham,AL,35294-0110,33.505697,-86.799345,POINT (-86.79935 33.50570)
2,100706,1097,253,0.230629,University of Alabama in Huntsville,Huntsville,AL,35899,34.724557,-86.640449,POINT (-86.64045 34.72456)
3,100751,1114,260,0.233393,The University of Alabama,Tuscaloosa,AL,35487-0100,33.211875,-87.545978,POINT (-87.54598 33.21187)
4,100858,2708,462,0.170606,Auburn University,Auburn,AL,36849,32.599378,-85.488258,POINT (-85.48826 32.59938)


In [19]:
geo_df.to_csv('geo_df2004.csv', index = False)